<a href="https://colab.research.google.com/github/randy-tsukemen/Data_science_roadmap/blob/master/shopee_test2_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import tensorflow as tf
import zipfile
import pandas as pd
import keras
import os
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import *
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [ ]:
train_path = '/content/drive/My Drive/shopee/train/train/'
test_path = '/content/drive/My Drive/shopee/test/test/'

In [ ]:
df = pd.read_csv("/content/drive/My Drive/shopee/train.csv")
df.sample(frac=1)
df.head()

,filename,category
0,45e2d0c97f7bdf8cbf3594beb6fdcda0.jpg,3
1,f74d1a5fc2498bbbfa045c74e3cc333e.jpg,3
2,f6c172096818c5fab10ecae722840798.jpg,3
3,251ffd610399ac00fea7709c642676ee.jpg,3
4,73c7328b8eda399199fdedec6e4badaf.jpg,3


In [ ]:
test_df = pd.read_csv("/content/drive/My Drive/shopee/test.csv")
test_df.head()

,filename,category
0,fd663cf2b6e1d7b02938c6aaae0a32d2.jpg,43
1,c7fd77508a8c355eaab0d4e10efd6b15.jpg,43
2,127f3e6d6e3491b2459812353f33a913.jpg,43
3,5ca4f2da11eda083064e6c36f37eeb81.jpg,43
4,46d681a542f2c71be017eef6aae23313.jpg,43


In [ ]:
df['category'] = df['category'].astype(int)
df['category'] = df['category'].apply(lambda x: "{:02d}".format(x)).astype(str)
cats = df['category']

In [ ]:
df['combined_filename'] = df['category'].map(lambda x: x + "/").astype(str) + df['filename']
df.head()

,filename,category,combined_filename
0,45e2d0c97f7bdf8cbf3594beb6fdcda0.jpg,03,03/45e2d0c97f7bdf8cbf3594beb6fdcda0.jpg
1,f74d1a5fc2498bbbfa045c74e3cc333e.jpg,03,03/f74d1a5fc2498bbbfa045c74e3cc333e.jpg
2,f6c172096818c5fab10ecae722840798.jpg,03,03/f6c172096818c5fab10ecae722840798.jpg
3,251ffd610399ac00fea7709c642676ee.jpg,03,03/251ffd610399ac00fea7709c642676ee.jpg
4,73c7328b8eda399199fdedec6e4badaf.jpg,03,03/73c7328b8eda399199fdedec6e4badaf.jpg


In [ ]:
mod_df = df.drop(columns = ["filename"])
mod_df.head()

,category,combined_filename
0,03,03/45e2d0c97f7bdf8cbf3594beb6fdcda0.jpg
1,03,03/f74d1a5fc2498bbbfa045c74e3cc333e.jpg
2,03,03/f6c172096818c5fab10ecae722840798.jpg
3,03,03/251ffd610399ac00fea7709c642676ee.jpg
4,03,03/73c7328b8eda399199fdedec6e4badaf.jpg


In [ ]:
mod_df = mod_df.sample(frac=1)
mod_df.head()

,category,combined_filename
14118,05,05/89f08db4c88936f18410d7e072c2693c.jpg
19521,20,20/1b6a821eafd8e572260732c0d70d5409.jpg
52837,38,38/1c3a716b702ae0dc183e91fb71dae051.jpg
77885,39,39/c8a2b3a30e623289a7f0d97462bc4c84.jpg
74322,06,06/5f04665c97c26804141ea30d203c9b3f.jpg


In [ ]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.1)
IMG_SIZE = 256

In [ ]:
train_generator=datagen.flow_from_dataframe(
dataframe=mod_df,
directory=train_path,
x_col="combined_filename",
y_col="category",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(IMG_SIZE,IMG_SIZE))

Found 94853 validated image filenames belonging to 42 classes.


In [ ]:
valid_generator=datagen.flow_from_dataframe(
dataframe=mod_df,
directory=train_path,
x_col="combined_filename",
y_col="category",
subset="validation",
batch_size=32,
seed=40,
shuffle=True,
class_mode="categorical",
target_size=(IMG_SIZE,IMG_SIZE))

Found 10539 validated image filenames belonging to 42 classes.


In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=test_df,
directory=test_path,
x_col="filename",
y_col=None,
batch_size=32,
seed=42,
shuffle=False,
class_mode=None,
target_size=(IMG_SIZE,IMG_SIZE))

Found 12185 validated image filenames.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
!pip install git+https://github.com/qubvel/efficientnet

  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-yoyr_tm0
  Running command git clone -q https://github.com/qubvel/efficientnet /tmp/pip-req-build-yoyr_tm0
  Created wheel for efficientnet: filename=efficientnet-1.1.0-cp36-none-any.whl size=18327 sha256=e95ee6b709bf7b0be7631e27e1290b98ed0d9cb4e1d8711ace898e99ca586179
  Stored in directory: /tmp/pip-ephem-wheel-cache-ycpi0gxy/wheels/64/60/2e/30ebaa76ed1626e86bfb0cc0579b737fdb7d9ff8cb9522663a
Successfully built efficientnet


In [ ]:
from efficientnet.keras import EfficientNetB7

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
# Create the base model from the pre-trained model EfficientNetB7
base_model = EfficientNetB7(input_shape=IMG_SHAPE,
                            include_top=False,
                            weights='imagenet')

258441216/258434480 [==============================] - 21s 0us/step


In [ ]:
base_model.summary()

Model: "efficientnet-b7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 128, 128, 64) 1728        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 128, 128, 64) 256         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 128, 128, 64) 0           stem_bn[0][0]                    
____________________________________________________________________________________

In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 650

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

Number of layers in the base model:  806


In [ ]:
global_average_layer = keras.layers.GlobalAveragePooling2D()
prediction_layer = keras.layers.Dense(42)

In [ ]:
new_model = keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [ ]:
from keras.optimizers import Adam
new_model.compile(Adam(lr=3e-4),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b7 (Model)      (None, 8, 8, 2560)        64097680  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2560)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 42)                107562    
Total params: 64,205,242
Trainable params: 37,760,426
Non-trainable params: 26,444,816
_________________________________________________________________


In [ ]:
mod_df.sample(frac=1)

,category,combined_filename
33152,28,28/672dbc363bd6018189914b0555142a69.jpg
75341,06,06/8f2fa1b02aac4cebacd53ad5a3d7b940.jpg
70076,08,08/1ad48fed934dc8a6b06180a4768b908a.jpg
42437,26,26/ccb5946c7fca3b7a63a4347a30048890.jpg
83679,24,24/9c1d0050de855b7305298fd11804794e.jpg
...,...,...
99937,14,14/fbbc468f710ef9046a3ffbdcdf57581c.jpg
4291,04,04/7753b6770597aa7f8e7091f0829716c3.jpg
2178,03,03/0e7506a6166636504ea1a6b768ee4668.jpg
40253,19,19/98d61652c6c4120216e32d744d9f1eee.jpg


In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size + 1
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size + 1
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size + 1
new_model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=4
)

Epoch 1/4
  14/2965 [..............................] - ETA: 15:06:07 - loss: 3.7035 - accuracy: 0.0491